In [53]:
from ipynb.fs.full.create_dataframes import *
import scipy.stats as stats
import numpy as np
import pandas as pd

In [62]:
def count_winning_pairs(sample_1, sample_2):
    sample_1, sample_2 = np.array(sample_1), np.array(sample_2)
    n_total_wins = 0
    for x in sample_1:
        n_wins = np.sum(x > sample_2) + 0.5*np.sum(x == sample_2)
        n_total_wins += n_wins
    return n_total_wins

def run_u_tests(spark, feature='danceability', alpha='.05'):
    feat_list = create_features_by_list(spark)
    pd_popular_lists = feat_list.toPandas()
    pd_less_lists = pd_popular_lists[(pd_popular_lists['followers'] > 1) & (pd_popular_lists['followers'] < 3)]
    pd_popular_lists = pd_popular_lists[(pd_popular_lists['followers'] >= 3 )]
    pd_less_lists = pd_less_lists.sort_values('followers',ascending=True)
    pd_popular_lists = pd_popular_lists.sort_values('followers',ascending=True)
    #create length variables to normalize
    poplistlen=np.shape(pd_popular_lists)[0]
    lesslistlen=np.shape(pd_less_lists)[0]
    lenlist = (poplistlen-lesslistlen)
    #normalizing list lengths
    if lenlist > 0:
        pd_less_lists = pd_less_lists.iloc[lenlist:,:].copy()
    elif lenlist == 0:
        pass
    else:
        pd_popular_lists = pd_popular_lists.iloc[lenlist:,:].copy()
    #run U_tests
    #sample_less_wins = count_winning_pairs(pd_less_lists.loc[:,feature],pd_popular_lists.loc[:,feature])
    #sample_popular_wins = count_winning_pairs(pd_popular_lists.loc[:,feature],pd_less_lists.loc[:,feature])
    
    res = stats.mannwhitneyu(pd_popular_lists.loc[:,feature], pd_less_lists.loc[:,feature], alternative="less")
    print("p-value for sample_popular_wins < sample_less_wins: {:2.3f}".format(res.pvalue))
    return res.pvalue

In [63]:
print(run_u_tests(spark))

p-value for sample_popular_wins < sample_less_wins: 0.147
0.14708826172483447


In [4]:
if __name__ == '__main__':
    spark = start_session()
    sc = add_context(spark)

In [42]:
if __name__ == '__main__':
    feat_list = create_features_by_list(spark)
    pd_popular_lists = feat_list.toPandas()
    pd_less_lists = pd_popular_lists[(pd_popular_lists['followers'] > 1) & (pd_popular_lists['followers'] < 3)]
    pd_popular_lists = pd_popular_lists[(pd_popular_lists['followers'] >= 3 )]
    pd_popular_lists.describe()

In [43]:
pd_less_lists = pd_less_lists.sort_values('followers',ascending=True)
pd_less_lists = pd_less_lists.iloc[19:,:]
pd_less_lists.describe()

,pid,followers,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,avg_tempo,valence,popularity
count,110.000000,110.0,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
mean,494.809091,2.0,0.253514,0.608351,0.626967,0.057318,0.193612,-7.529372,0.093901,121.027248,0.496539,55.521041
std,295.820194,0.0,0.153325,0.082154,0.113173,0.111273,0.042610,2.454975,0.050340,6.048401,0.107600,7.092476
min,8.000000,2.0,0.044072,0.329957,0.204278,0.000001,0.118324,-20.185657,0.031727,95.316308,0.123307,33.696960
25%,212.000000,2.0,0.156455,0.568071,0.595866,0.005311,0.174975,-8.138493,0.059308,117.687049,0.452970,52.428229
50%,513.500000,2.0,0.215384,0.614166,0.648568,0.026094,0.188700,-6.809260,0.076027,120.499659,0.489537,57.036270
75%,745.250000,2.0,0.273855,0.672862,0.692288,0.059981,0.200090,-6.091859,0.109990,123.929208,0.549702,59.980108
max,998.000000,2.0,0.888542,0.751803,0.839896,0.744708,0.497137,-4.588458,0.250473,141.679616,0.860603,69.036683


In [55]:
np.shape(pd_popular_lists)

(110, 13)